# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as mp
import pandas as pd
import requests as req
import xlsxwriter
import math

ModuleNotFoundError: No module named 'xlsxwriter'

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
#We are importing the list of stocks but the list we are using is static meanining. Not dynamic and directly linked to S&P 500 info source
stocks = pd.read_csv("sp_500_stocks.csv") 
stocks


NameError: name 'pd' is not defined

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [6]:
#Remember to keep API tokens in secrets.py and add it to GitIgnore
#You might need to refresh the notebook after calling this
from secrets import IEX_CLOUD_API_TOKEN
from secrets import ALPHA_VANTAGE_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [7]:

#We are getting those two features for our stocks
#We are using a sandbox API and geting randomized data
symbol = 'AAPL'


# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}' 

#The f there is a fancy way of concatenating strings and a variable holding a string
#We got the base url from the iex cloud documentation
#The part strating from stock until quote/ was collected from the documentation as well.
# marketcap: Market capital of the security calculated as shares outstanding * previous day close
#We had to add /stable/ before stocks because there was naming convention. You have to mention if you want the stable api or the latest one


#IEX has discontinued their sandbox API service so we are going to use  Alpha Vantage 
#But all our required endpoints are not in one place so we'll get it from different endpoints and 
#stitch them together in the dataframe we are using.


#The fields each stock will have in the pandas dataframe: ticker, market cap, price, number of shares to buy 

#print(api_url)

#data = req.get(api_url).json()
#print(data)

#Commented the last two lines out after recording the following 3 log entries
#We got 404 first with IEX because we did not add /stable/
#We got 403 (forbidden access error) on the second try (thats when i realized the problem with IEX).
#We got 200 on the third try (first try with Alpha Vantage)
api_url_for_marketCap = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}"
dataForMarketCap = req.get(api_url_for_marketCap).json()
print(dataForMarketCap)

#We change the functions for the different endpoints we want
api_url_for_price = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}"
globalQuote = req.get(api_url_for_price).json()
#The whole dictionary that contains the price value is inside another dictionary called global quote.
#Hence the extra step
dataForPrice = globalQuote["Global Quote"]
print(dataForPrice)




{'Symbol': 'AAPL', 'AssetType': 'Common Stock', 'Name': 'Apple Inc', 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.", 'CIK': '320193', 'Exchange': 'NASDAQ', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'TECHNOLOGY', 'Industry': 'ELECTRONIC COMPUTERS', 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US', 'FiscalYearEnd': 'September', 'LatestQuarter': '2023-03-31', 'MarketCapitalization': '2698415702000', 'EBITDA': '123788001000', 'PERatio': '29.03', 'PEGRatio': '2.75', 'BookValue': '3.953', 'Di

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [8]:
marketCap = dataForMarketCap["MarketCapitalization"]

#print(marketCap)
price = dataForPrice["05. price"]
#print(price)

#These produced expected outputs

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [9]:
myColumns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
#We already have marketCap and price data from the API. 
#Ticker is basically the stock code (AAPL) and number of shares to buy is gonna be calculated

#finalDataframe = pd.DataFrame([[0,0,0,0]],columns = myColumns) 
#For the first parammeter, the outer [] is for the mother list that will have all the data. 
# So every list in the mother list is a record or a row

finalDataframe = pd.DataFrame(columns = myColumns)
finalDataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [10]:
"""
finalDataframe.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A' #We use N/A here because we haven't calculated it yet
        ],
        index = myColumns #This line tells the append method which colums to put the data under
        
    ), ignore_index = True #We always need to add this to the append method
)
"""

"\nfinalDataframe.append(\n    pd.Series(\n        [\n            symbol,\n            price,\n            marketCap,\n            'N/A' #We use N/A here because we haven't calculated it yet\n        ],\n        index = myColumns #This line tells the append method which colums to put the data under\n        \n    ), ignore_index = True #We always need to add this to the append method\n)\n"

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

## Switching to YFinance

So far we have worked with Alpha Vantage. But in the next step I realized that Alpha Vantage is not enough as it is limited to only 5 calls per minute. Let us try Yfinance now.

In [11]:
"""
finalDataframe = pd.DataFrame(columns = myColumns)

for stock in stocks['Ticker']:
    print(stock)
    #Everything we did for the AAPL stock so far, we just copy-paste here and do it for all the stocks
    api_url_for_price = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}"
    globalQuote = req.get(api_url_for_price).json()
    #dataForPrice = globalQuote["Global Quote"]
    #price = dataForPrice["05. price"]
    api_url_for_marketCap = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}"
    dataForMarketCap = req.get(api_url_for_marketCap).json()
    #marketCap = dataForMarketCap['MarketCapitalization']
    print(dataForMarketCap)
    print(globalQuote)
    
    finalDataframe.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A' #We use N/A here because we haven't calculated it yet
        ],
        index = myColumns #This line tells the append method which colums to put the data under
        
    ), ignore_index = True #We always need to add this to the append method
    )
finalDataframe
"""
    
    

'\nfinalDataframe = pd.DataFrame(columns = myColumns)\n\nfor stock in stocks[\'Ticker\']:\n    print(stock)\n    #Everything we did for the AAPL stock so far, we just copy-paste here and do it for all the stocks\n    api_url_for_price = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}"\n    globalQuote = req.get(api_url_for_price).json()\n    #dataForPrice = globalQuote["Global Quote"]\n    #price = dataForPrice["05. price"]\n    api_url_for_marketCap = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_TOKEN}"\n    dataForMarketCap = req.get(api_url_for_marketCap).json()\n    #marketCap = dataForMarketCap[\'MarketCapitalization\']\n    print(dataForMarketCap)\n    print(globalQuote)\n    \n    finalDataframe.append(\n    pd.Series(\n        [\n            symbol,\n            price,\n            marketCap,\n            \'N/A\' #We use N/A here because we haven\'t calculated it yet\n 

In [12]:
#Importing the package
import yfinance as yf
#symbol = "AAPL"
#data = yf.Ticker(symbol).info
#print(data)
#marketCap = data['marketCap']
#price = data['currentPrice']

In [13]:
#Making API call and parsing for marketCap and currentPrice

#This file had the latest list of S&P 500 stocks I could find as of 24 May 2023. 
#If you get error in this section then just print out the headings and see
#which stock gets you the error. Probably that stock isn't in the list anymore.

#Had to get rid off BF.B and BRK.B from the constituent list because YFinance did not have complete data on them for some reason


stocks = pd.read_csv("constituents.csv")
finalDataframe = pd.DataFrame(columns = myColumns)
for stock in stocks["Ticker"]:
    print(stock)
    data = yf.Ticker(stock).info
    #print(data)
    marketCap = data['marketCap']
    price = data['currentPrice']
    
    finalDataframe = finalDataframe.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A' #We use N/A here because we haven't calculated it yet
        ],
        index = myColumns #This line tells the append method which colums to put the data under
        
    ), ignore_index = True #We always need to add this to the append method
    )
"""
    


'\nstocks = pd.read_csv("constituents.csv")\nfinalDataframe = pd.DataFrame(columns = myColumns)\nfor stock in stocks["Ticker"]:\n    print(stock)\n    data = yf.Ticker(stock).info\n    #print(data)\n    marketCap = data[\'marketCap\']\n    price = data[\'currentPrice\']\n    \n    finalDataframe = finalDataframe.append(\n    pd.Series(\n        [\n            symbol,\n            price,\n            marketCap,\n            \'N/A\' #We use N/A here because we haven\'t calculated it yet\n        ],\n        index = myColumns #This line tells the append method which colums to put the data under\n        \n    ), ignore_index = True #We always need to add this to the append method\n    )\n'

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [16]:
#This splits the whole list of stocks into chunks
def chunks(lst,n):
    for i in range(0, len(lst),n):
        yield lst[i : i+n]
#Check out what the yield keyword does online. It basically returns values as an object or a chunk

In [17]:
symbolGroups = list(chunks(stocks["Ticker"],100))
symbolStrings = []
#which stock gets you the error. Probably that stock isn't in the list anymore.

#Had to get rid off BF.B and BRK.B from the constituent list because YFinance did not have complete data on them for some reason
stocks = pd.read_csv("constituents.csv")
for i in range(0, len(symbolGroups)):
    symbolStrings.append(",".join(symbolGroups[i]))
finalDataframe = pd.DataFrame(columns = myColumns)


for symbolString in symbolStrings:
    #print(symbolString)
    batchCall = yf.Tickers(symbolString)
    for symbol in symbolString.split(","):
        data = batchCall.tickers[symbol].info
        marketCap = data['marketCap']
        price = data['currentPrice'] 
        finalDataframe = finalDataframe.append(
        pd.Series(
            [
                symbol,
                price,
                marketCap,
                'N/A' #We use N/A here because we haven't calculated it yet
            ],
            index = myColumns #This line tells the append method which colums to put the data under
            
        ), ignore_index = True #We always need to add this to the append method
        ) 
finalDataframe
#1m 18s this is not actually a noticeable improvement from the previous implementation
#Which means we are not actually doing a batch api call.
#Look into it later

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,99.015,54623801344,N/A
1,AOS,66.385,9987092480,N/A
2,ABT,104.650,181981118464,N/A
3,ABBV,142.320,251093778432,N/A
4,ACN,286.440,181041545216,N/A
...,...,...,...,...
496,YUM,130.660,36596170752,N/A
497,ZBRA,266.305,13696145408,N/A
498,ZBH,129.323,27131705344,N/A
499,ZION,28.425,4209770496,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [18]:

while(True):
    portfolioSize = input("Enter the value of your portfolio: ")
    try:
        val = float(portfolioSize)
        print(val)
        break
    except ValueError:
        print("Did you input a number?\nPlease input a number.")
finalDataframe

1000000.0


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,99.015,54623801344,N/A
1,AOS,66.385,9987092480,N/A
2,ABT,104.650,181981118464,N/A
3,ABBV,142.320,251093778432,N/A
4,ACN,286.440,181041545216,N/A
...,...,...,...,...
496,YUM,130.660,36596170752,N/A
497,ZBRA,266.305,13696145408,N/A
498,ZBH,129.323,27131705344,N/A
499,ZION,28.425,4209770496,N/A


In [19]:
print(len(finalDataframe.index))
finalDataframe

501


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,99.015,54623801344,N/A
1,AOS,66.385,9987092480,N/A
2,ABT,104.650,181981118464,N/A
3,ABBV,142.320,251093778432,N/A
4,ACN,286.440,181041545216,N/A
...,...,...,...,...
496,YUM,130.660,36596170752,N/A
497,ZBRA,266.305,13696145408,N/A
498,ZBH,129.323,27131705344,N/A
499,ZION,28.425,4209770496,N/A


In [20]:
positionSize = val/len(finalDataframe.index)
#number_of_apple_shares = positionSize/500
#The result might be a float and we are not allowed to buy fractions of stocks
#So we must round down because if we round up or round randomly we miht end up with more shares than we have money for
#print(math.floor(number_of_apple_shares))

for i in range(0, len(finalDataframe.index)):
    finalDataframe.loc[i, "Number of Shares to Buy"] = math.floor(positionSize/finalDataframe.loc[i,"Stock Price"])
finalDataframe 

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,99.015,54623801344,20
1,AOS,66.385,9987092480,30
2,ABT,104.650,181981118464,19
3,ABBV,142.320,251093778432,14
4,ACN,286.440,181041545216,6
...,...,...,...,...
496,YUM,130.660,36596170752,15
497,ZBRA,266.305,13696145408,7
498,ZBH,129.323,27131705344,15
499,ZION,28.425,4209770496,70


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [39]:
writer = pd.ExcelWriter("recommended_trades.xlsx", engine = 'xlsxwriter')
finalDataframe.to_excel(writer, "Recommended Trades", index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [40]:
background_color = "#191970"
font_color = "#ffffff"
string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1

    }
)
dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1

    }
)
integer_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1

    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [37]:
writer.sheets["Recommended Trades"].set_column("A:A",18,string_format)
#We had to re-initialize after the first formatting
writer.sheets["Recommended Trades"].set_column("B:B",18,string_format)
writer.sheets["Recommended Trades"].set_column("C:C",18,string_format)
writer.sheets["Recommended Trades"].set_column("D:D",18,string_format)

writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [44]:
column_formats = {
    "A": ["Ticker", string_format],
    "B": ["Stock Price", dollar_format],
    "C": ["Market Capitalization", dollar_format],
    "D": ["Number of Shares to Buy", integer_format]
}
header_format = {
    "font_color": "#FFA500",
    "bg_color": "#710193",
    "border": 1
}


for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f"{column}:{column}", 20, column_formats[column][1])
    writer.sheets["Recommended Trades"].write(f"{column}1", column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [45]:
writer.save()

c:\Users\Siam\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
